In [ ]:
!pip install ipython==7.34.0 ipykernel==5.5.6
!pip install import_ipynb

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter, zoom
from skimage.filters import threshold_otsu
from sklearn.metrics import mean_squared_error
from skimage.metrics import structural_similarity as ssim

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

from google.colab import drive
drive.mount('/content/drive')
import os
import import_ipynb
import imageio

path = '/content/drive/MyDrive/Colab Notebooks/Physics-Informed Neural Networks/Demo/fenics_cfd/neural_network'
os.chdir(path)

from visualize_geometry import plot_numpy_matrices

Mounted at /content/drive
importing Jupyter notebook from visualize_geometry.ipynb


In [ ]:
def to_numpy(tensor):
  return tensor.detach().cpu().numpy()

In [ ]:
def create_meshgrid(height, width, device, num_sampled_points=None):

  y_coords, x_coords = torch.meshgrid(torch.linspace(0, height, 280), torch.linspace(0, width, 280), indexing='xy')
  coords = torch.stack([x_coords, y_coords], dim=0)

  if num_sampled_points == None:
    coords = coords.to(device)
    return coords.requires_grad_(True)

  flat_coords = coords.view(2, -1).transpose(0, 1)  # Shape: [280*280, 2]

  indices = torch.randperm(flat_coords.size(0))[:num_sampled_points]
  sampled_flat_coords = torch.zeros_like(flat_coords)
  sampled_flat_coords[indices] = flat_coords[indices]

  sampled_coords = sampled_flat_coords.transpose(0, 1).view(2, 280, 280).unsqueeze(0)
  sampled_coords = sampled_coords.to(device)

  return sampled_coords.requires_grad_(True)

In [ ]:
def segment_vessel_otsu(velocity_field, pressure_field, gamma, sigma, target_size, device):
  def apply_blur(field, gamma, sigma):
    field_transformed = np.maximum(field, 0) ** gamma
    return gaussian_filter(field_transformed / np.max(field_transformed), sigma=sigma)

  def upscale(field, target_size):
    zoom_factors = [target_size[i] / field.shape[i+2] for i in range(2)]
    return zoom(field, zoom=[1, 1, *zoom_factors], order=3)  # Bicubic interpolation

  velocity_np = to_numpy(velocity_field)
  pressure_np = to_numpy(pressure_field)

  if velocity_np.shape[2:] != target_size:
    velocity_np = upscale(velocity_np, target_size)

  if pressure_np.shape[2:] != target_size:
    pressure_np = upscale(pressure_np, target_size)

  velocity_magnitude = np.linalg.norm(velocity_np, axis=1)
  blurred_velocity = apply_blur(velocity_magnitude, gamma, sigma)
  blurred_pressure = apply_blur(pressure_np.squeeze(1), gamma, sigma)

  velocity_mask = (blurred_velocity > threshold_otsu(blurred_velocity)).astype(np.float32)
  pressure_mask = (blurred_pressure > threshold_otsu(blurred_pressure)).astype(np.float32)

  combined_mask = np.maximum(velocity_mask[:, np.newaxis, :, :], pressure_mask[:, np.newaxis, :, :])

  return torch.from_numpy(combined_mask).float().to(device)

In [ ]:
def compute_navier_stokes_loss(u_pred, p_pred, coordinates, mask, rho, mu, device):
  u, v, p = u_pred[:, [0]] * mask, u_pred[:, [1]] * mask, p_pred * mask

  du_dxy = torch.autograd.grad(u, coordinates, grad_outputs=torch.ones_like(u).to(device), retain_graph=True, create_graph=True)[0]
  dv_dxy = torch.autograd.grad(v, coordinates, grad_outputs=torch.ones_like(v).to(device), retain_graph=True, create_graph=True)[0]
  dp_dxy = torch.autograd.grad(p, coordinates, grad_outputs=torch.ones_like(p).to(device), retain_graph=True, create_graph=True)[0]

  d2u_dxy2 = torch.autograd.grad(du_dxy, coordinates, grad_outputs=torch.ones_like(du_dxy).to(device), create_graph=True)[0]
  d2v_dxy2 = torch.autograd.grad(dv_dxy, coordinates, grad_outputs=torch.ones_like(dv_dxy).to(device), create_graph=True)[0]

  du_dx, du_dy = du_dxy[:, [0]], du_dxy[:, [1]]
  dv_dx, dv_dy = dv_dxy[:, [0]], dv_dxy[:, [1]]
  dp_dx, dp_dy = dp_dxy[:, [0]], dp_dxy[:, [1]]
  d2u_dx2, d2u_dy2 = d2u_dxy2[:, [0]], d2u_dxy2[:, [1]]
  d2v_dx2, d2v_dy2 = d2v_dxy2[:, [0]], d2v_dxy2[:, [1]]

  continuity = du_dx + dv_dy
  continuity_loss = torch.max(torch.abs(continuity))

  momentum_u = rho * (u * du_dx + v * du_dy) + dp_dx - mu * (d2u_dx2 + d2u_dy2)
  momentum_v = rho * (u * dv_dx + v * dv_dy) + dp_dy - mu * (d2v_dx2 + d2v_dy2)
  momentum_loss = torch.max(torch.abs(momentum_u)) + torch.max(torch.abs(momentum_v))

  physics_loss = continuity_loss + momentum_loss

  return physics_loss / 1000

In [ ]:
def train_model(model, train_loader, optimizer, criterion, num_epochs, device, alpha=0.5, save_path=None, is_physics_informed=False):
  if device.type == 'cuda':
    torch.cuda.empty_cache()
  model.train()

  all_data_losses = []
  all_physics_losses = []

  if save_path is not None:
    os.makedirs(save_path, exist_ok=True)
    save_file = os.path.join(save_path, 'model_params.pt')

  for epoch in range(num_epochs):
    running_loss = 0.0
    running_physics_loss = 0.0

    for i, ((u_hr, p_hr), (u_lr, p_lr)) in enumerate(train_loader):
      u_hr, p_hr = u_hr.to(device), p_hr.to(device)
      u_lr, p_lr = u_lr.to(device), p_lr.to(device)

      optimizer.zero_grad()

      if is_physics_informed:
        coordinates = create_meshgrid(0.006, 0.006, device, num_sampled_points=1000).requires_grad_(True)
        coordinates = coordinates.expand(u_lr.size(0), -1, -1, -1)
        u_pred, p_pred = model(u_lr, p_lr, coordinates)
        mask = segment_vessel_otsu(u_lr, p_lr, gamma=0.5, sigma=1.5, target_size=(280, 280), device=device)
        data_loss = (criterion(u_pred, u_hr) + criterion(p_pred, p_hr))
        physics_loss = compute_navier_stokes_loss(u_pred, p_pred, coordinates, mask, rho=1060, mu=0.0035, device=device)
        loss = (1 - alpha) * data_loss + alpha * physics_loss
      else:
        u_pred, p_pred = model(u_lr, p_lr)
        loss = (criterion(u_pred, u_hr) + criterion(p_pred, p_hr))

      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      if is_physics_informed:
        running_physics_loss += physics_loss

      save_interval=1000
      if save_path is not None and (epoch * len(train_loader) + i) % save_interval == 0:
        torch.save(model.state_dict(), save_file)

      if (i + 1) % 10 == 0:
        if is_physics_informed:
          print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / 10:.6f}, Physics Loss: {running_physics_loss / 10:.6f}')
        else:
          print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / 10:.6f}')
          all_physics_losses.append(running_physics_loss / 10)

        running_loss = 0.0
        running_physics_loss = 0.0

    if save_path is not None:
      torch.save(model.state_dict(), save_file)

  return all_data_losses, all_physics_losses if is_physics_informed else all_data_losses

In [ ]:
def test_model(model, test_loader, criterion, device, num_visualizations=3, is_physics_informed=False):
  model.eval()
  total_loss = 0
  results = []

  with torch.no_grad():
    for i, ((u_hr, p_hr), (u_lr, p_lr)) in enumerate(test_loader):
      u_hr, p_hr = u_hr.to(device), p_hr.to(device)
      u_lr, p_lr = u_lr.to(device), p_lr.to(device)

      if is_physics_informed:
        coordinates = create_meshgrid(0.006, 0.006, device, num_sampled_points=1000)
        coordinates = coordinates.expand(u_lr.size(0), -1, -1, -1)
        u_pred, p_pred = model(u_lr, p_lr, coordinates)
      else:
        u_pred, p_pred = model(u_lr, p_lr)

      total_loss += (criterion(u_pred, u_hr) + criterion(p_pred, p_hr)).item()

      results.append({
        'noisy': (u_lr.cpu().numpy(), p_lr.cpu().numpy()),
        'predicted': (u_pred.cpu().numpy(), p_pred.cpu().numpy()),
        'true': (u_hr.cpu().numpy(), p_hr.cpu().numpy())
      })

      if i < num_visualizations:
        u_pred_np, p_pred_np = to_numpy(u_pred), to_numpy(p_pred)
        u_hr_np, p_hr_np = to_numpy(u_hr), to_numpy(p_hr)

        for j in range(u_pred_np.shape[0]):
          plot_numpy_matrices(u_hr_np[j].transpose(1, 2, 0), p_hr_np[j][0], main_title="True Flow Field", plot_size=6)
          plot_numpy_matrices(u_pred_np[j].transpose(1, 2, 0), p_pred_np[j][0], main_title="Predicted Flow Field", plot_size=6)

  avg_loss = total_loss / len(test_loader)
  print(f'Test Loss: {avg_loss}')

  return avg_loss, results

In [ ]:
def evaluate_model(model, test_loader, device, is_physics_informed=False):
  model.eval()

  def psnr(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    max_pixel = 1.0  # Assuming pixel values range from 0 to 1
    return 20 * np.log10(max_pixel / np.sqrt(mse))

  total_mse_velocity = total_psnr_velocity = total_ssim_velocity = 0
  total_mse_pressure = total_psnr_pressure = total_ssim_pressure = 0
  num_samples = len(test_loader.dataset)

  with torch.no_grad():
    for (u_hr, p_hr), (u_lr, p_lr) in test_loader:
      u_hr, p_hr = u_hr.to(device), p_hr.to(device)
      u_lr, p_lr = u_lr.to(device), p_lr.to(device)

      if is_physics_informed:
        coordinates = create_meshgrid(0.006, 0.006, device, num_sampled_points=1000)
        u_pred, p_pred = model(u_lr, p_lr, coordinates)
      else:
        u_pred, p_pred = model(u_lr, p_lr)

      u_hr_np, u_pred_np = to_numpy(u_hr).squeeze(0), to_numpy(u_pred).squeeze(0)
      p_hr_np, p_pred_np = to_numpy(p_hr).squeeze(0).squeeze(0), to_numpy(p_pred).squeeze(0).squeeze(0)

      for i in range(u_hr_np.shape[0]):
        total_mse_velocity += mean_squared_error(u_hr_np[i], u_pred_np[i])
        total_psnr_velocity += psnr(u_hr_np[i], u_pred_np[i])
        total_ssim_velocity += ssim(u_hr_np[i], u_pred_np[i])

        total_mse_pressure += mean_squared_error(p_hr_np[i], p_pred_np[i])
        total_psnr_pressure += psnr(p_hr_np[i], p_pred_np[i])
        total_ssim_pressure += ssim(p_hr_np[i], p_pred_np[i])

  avg_mse_velocity = total_mse_velocity / len(test_loader)
  avg_psnr_velocity = total_psnr_velocity / len(test_loader)
  avg_ssim_velocity = total_ssim_velocity / len(test_loader)

  avg_mse_pressure = total_mse_pressure / len(test_loader)
  avg_psnr_pressure = total_psnr_pressure / len(test_loader)
  avg_ssim_pressure = total_ssim_pressure / len(test_loader)

  print(f"Velocity - Average MSE: {avg_mse_velocity:.4f}, Average PSNR: {avg_psnr_velocity:.4f} dB, Average SSIM: {avg_ssim_velocity:.4f}")
  print(f"Pressure - Average MSE: {avg_mse_pressure:.4f}, Average PSNR: {avg_psnr_pressure:.4f} dB, Average SSIM: {avg_ssim_pressure:.4f}")